In [16]:
import torch
# Check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
# Check if GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")
save_directory = "/home/azureuser/cloudfiles/code/Users/shahriarizadehfatima"
import os
from PIL import Image, ImageDraw, ImageFont, ImageOps
import numpy as np
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch.nn.functional as F
from torchvision.transforms import functional as F_transform
from torchvision.transforms import Normalize as NormalizeTransform
from torchvision.ops.boxes import box_iou
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from albumentations import Compose, Resize, HorizontalFlip, BboxParams, VerticalFlip, Normalize
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import random
from azureml.core import Dataset, Datastore, Workspace
from torchmetrics.detection import MeanAveragePrecision
from datetime import datetime
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

Using device: cuda


/anaconda/envs/Object_Detection/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
batch_size = 8
num_workers = 3

In [17]:
json_dir = "/home/azureuser/cloudfiles/code/Users/shahriarizadehfatima/config.json"
# Define folders to exclude
exclude_folders = {}
# Load workspace from config.json
ws = Workspace.from_config(json_dir)
datastore = Datastore.get(ws, "fasteners_1")
dataset_images = Dataset.File.from_files(path=(datastore, 'images/'))
mounted_context_images = dataset_images.mount()
mounted_context_images.start()
# Get the mount path
images_dir = mounted_context_images.mount_point
images_dir_folders = [folder for folder in os.listdir(images_dir) if folder not in exclude_folders]
#dataset_labels
dataset_labels = Dataset.File.from_files(path=(datastore, 'labels/'))
mounted_context_labels = dataset_labels.mount()
mounted_context_labels.start()
# Get the mount path
labels_dir = mounted_context_labels.mount_point
labels_dir_folders = [folder for folder in os.listdir(labels_dir) if folder not in exclude_folders]
classes_file = labels_dir_folders

Available datastores: ['azureml_globaldatasets', 'fasteners_1', 'workspacefilestore', 'workspaceworkingdirectory', 'workspaceartifactstore', 'workspaceblobstore']
Images Dataset mounted at: /tmp/tmpy9owx32p ['Cable Straps Edge Mount', 'Cable Straps Hole Mount', 'Tape-on Locators Edge Mount', 'Tape-on Locators Hole Mount']
Labels Dataset mounted at: /tmp/tmphayze_bw ['Cable Straps Edge Mount', 'Cable Straps Hole Mount', 'Tape-on Locators Edge Mount', 'Tape-on Locators Hole Mount']
classes_file: ['Cable Straps Edge Mount', 'Cable Straps Hole Mount', 'Tape-on Locators Edge Mount', 'Tape-on Locators Hole Mount']


In [18]:
# Get the list of class names (subdirectories in images/)
class_names = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
num_classes = len(class_names)+ 1

# Lists to store file paths for each split
train_files = []
test_files = []
val_files = []

# Split the data and store file paths
for class_name in class_names:
    image_class_dir = os.path.join(images_dir, class_name)
    label_class_dir = os.path.join(labels_dir, class_name)
    image_files = [f for f in os.listdir(image_class_dir)]

    num_images = len(image_files)
    train_split = int(train_ratio * num_images)
    test_split = int(test_ratio * num_images)

    train_image_files = image_files[:train_split]
    test_image_files = image_files[train_split:train_split + test_split]
    val_image_files = image_files[train_split + test_split:]

    for split, files in zip(['train', 'test', 'val'], [train_image_files, test_image_files, val_image_files]):
        for file_name in files:
            image_path = os.path.join(image_class_dir, file_name)
            label_path = os.path.join(label_class_dir, file_name.replace(os.path.splitext(file_name)[1], '.txt'))
            if split == 'train':
                train_files.append((image_path, label_path, class_name))
            elif split == 'test':
                test_files.append((image_path, label_path, class_name))
            elif split == 'val':
                val_files.append((image_path, label_path, class_name))

len tarain: 275
len tarain: 77


In [19]:
class CustomDataset(Dataset):
    def __init__(self, file_list, classes_file, image_size=(224, 224), augmentation=False):
        self.file_list = file_list
        self.classes = list_subfolders(classes_file)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.new_width, self.new_height = image_size
        self.resize = Resize(height=self.new_height, width=self.new_width)
        self.normalize = NormalizeTransform(mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225])
        self.augmentations = get_augmentations() if augmentation else []

    def __len__(self):
        return len(self.file_list) * (len(self.augmentations) + 1)

    def __getitem__(self, idx):
        img_idx = idx // (len(self.augmentations) + 1)
        aug_idx = idx % (len(self.augmentations) + 1)
        img_path, label_path, _ = self.file_list[img_idx]

        image = Image.open(img_path).convert("RGB")
        img = np.array(image)
        img = self.resize(image=img)['image']

        bboxes, labels = [], []
        with open(label_path, "r") as f:
            for line in f:
                if line.startswith("Class:"):
                    class_name = line.split(":")[1].strip().strip('"')
                elif line.startswith("Bounding Box:"):
                    bbox = eval(line.split(":")[1].strip())
                    bboxes.append(bbox)
                    labels.append(self.classes.index(class_name) + 1)

        if not bboxes:
            return self.__getitem__((idx + 1) % len(self))

        # Scale bboxes
        orig_width, orig_height = image.size
        scale_x, scale_y = self.new_width / orig_width, self.new_height / orig_height
        bboxes = [[x1 * scale_x, y1 * scale_y, x2 * scale_x, y2 * scale_y] for x1, y1, x2, y2 in bboxes]

        if aug_idx != 0:
            aug = self.augmentations[aug_idx - 1]
            augmented = aug(image=img, bboxes=bboxes, class_labels=labels)
            img = augmented['image']
            bboxes = augmented['bboxes']
            labels = augmented['class_labels']
            img = torch.from_numpy(img.astype(np.float32) / 255.0).permute(2, 0, 1)
        else:
            img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0

        img = self.normalize(img)
        return idx, img, {"boxes": torch.tensor(bboxes, dtype=torch.float32),
                          "labels": torch.tensor(labels, dtype=torch.int64)}
def list_subfolders(main_folder):
    if isinstance(main_folder, list):
        return main_folder
    if not os.path.isdir(main_folder):
        raise ValueError(f"The provided path '{main_folder}' is not valid.")
    return [name for name in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, name))]

def custom_collate(batch):
    images, targets = zip(*batch)
    return torch.stack(images, 0), list(targets)

def get_augmentations():
    bbox_params = BboxParams(format="pascal_voc", label_fields=["class_labels"])
    return [
        Compose([HorizontalFlip()], bbox_params=bbox_params),
        Compose([VerticalFlip()], bbox_params=bbox_params)
    ]

class NormalizeTransform:
    def __init__(self, mean, std):
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def __call__(self, tensor):
        return (tensor - self.mean) / self.std

def denormalize(tensor, mean, std):
    mean = torch.tensor(mean).view(-1, 1, 1)
    std = torch.tensor(std).view(-1, 1, 1)
    return tensor * std + mean

transform = ToTensorV2()

/anaconda/envs/Object_Detection/lib/python3.10/site-packages/albumentations/core/composition.py:250: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [20]:
# Create Dataset instances
train_dataset = CustomDataset(file_list = train_files, classes_file=  classes_file , augmentation= Augmentation, image_size=(224,224))
test_dataset = CustomDataset(file_list= test_files, classes_file=  classes_file)
val_dataset = CustomDataset(file_list= val_files, classes_file=  classes_file)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers= num_workers, shuffle=True, collate_fn=lambda batch: tuple(zip(*batch)))
test_loader = DataLoader(test_dataset, batch_size=batch_size,  num_workers= num_workers, shuffle=True, collate_fn=lambda batch: tuple(zip(*batch)))
val_loader = DataLoader(val_dataset, batch_size=batch_size,  num_workers= num_workers, shuffle=True, collate_fn=lambda batch: tuple(zip(*batch)))

Number of training images: 275
Number of validation images: 77


In [ ]:
class ModelTrainer:
    def __init__(
        self,
        backbone_name="resnet50",
        pretrained=True,
        num_classes=2,
        learning_rate=0.005,
        weight_decay=0.0005,
        num_epochs=10,
        detections_per_img=100,
        rpn_nms_thresh=0.5,
        rpn_score_thresh=0.4,
        device="cuda" if torch.cuda.is_available() else "cpu"
    ):
        """
        Initialize the trainer with model and training configurations.

        Args:
            backbone_name (str): Name of the backbone (e.g., 'resnet34', 'resnet50', 'resnet101').
            pretrained (bool): Whether to use a pretrained backbone.
            num_classes (int): Number of classes (including background).
            learning_rate (float): Initial learning rate for SGD optimizer.
            weight_decay (float): Weight decay for regularization.
            num_epochs (int): Number of training epochs.
            detections_per_img (int): Max detections per image.
            rpn_nms_thresh (float): NMS threshold for RPN proposals.
            rpn_score_thresh (float): Score threshold for RPN proposals.
            device (str): Device to train on ('cuda' or 'cpu').
        """
        self.backbone_name = backbone_name
        self.pretrained = pretrained
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.num_epochs = num_epochs
        self.detections_per_img = detections_per_img
        self.rpn_nms_thresh = rpn_nms_thresh
        self.rpn_score_thresh = rpn_score_thresh
        self.device = device

        self.model = self._build_model()
        self.optimizer = torch.optim.SGD(
            self.model.parameters(),
            lr=self.learning_rate,
            momentum=0.9,
            weight_decay=self.weight_decay
        )
        self.lr_scheduler = ReduceLROnPlateau(
            self.optimizer,
            mode="min",
            factor=0.1,
            patience=5,
            verbose=True
        )
        self.scaler = torch.cuda.amp.GradScaler()
        self.metric = MeanAveragePrecision()
        self.model.to(self.device)

    def _build_model(self):
        """Set up the Faster R-CNN model with custom configurations."""
        # Define backbone
        backbone = resnet_fpn_backbone(self.backbone_name, pretrained=self.pretrained)

        # Custom anchor sizes and aspect ratios
        anchor_sizes = ((32,), (64,), (128,), (140,), (224,))
        aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
        custom_anchor_generator = AnchorGenerator(
            sizes=anchor_sizes,
            aspect_ratios=aspect_ratios
        )

        # Initialize Faster R-CNN model
        model = FasterRCNN(
            backbone,
            num_classes=self.num_classes,
            rpn_pre_nms_top_n_train=100,
            rpn_post_nms_top_n_train=100,
            rpn_nms_thresh=self.rpn_nms_thresh,
            rpn_score_thresh=self.rpn_score_thresh,
            rpn_anchor_generator=custom_anchor_generator,
            box_fg_iou_thresh=0.5,
            box_bg_iou_thresh=0.5
        )

        # Replace the box predictor
        representation_size = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(
            representation_size,
            self.num_classes
        )
        model.roi_heads.detections_per_img = self.detections_per_img

        return model

    def train_one_epoch(self, train_loader, epoch):
        """Train the model for one epoch."""
        self.model.train()
        train_loss = 0.0
        train_loader_progress = tqdm(
            train_loader,
            desc=f"Training Epoch {epoch + 1}",
            unit="batch"
        )

        for images, targets in train_loader_progress:
            images = [img.to(self.device) for img in images]
            targets = [{k: v.to(self.device) for k, v in t.items()} for t in targets]

            self.optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                loss_dict = self.model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                train_loss += losses.item()

            self.scaler.scale(losses).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()

            train_loader_progress.set_postfix(loss=f"{losses.item():.4f}")

        avg_train_loss = train_loss / len(train_loader)
        return avg_train_loss

    def validate_one_epoch(self, val_loader, epoch):
        """Validate the model for one epoch."""
        self.model.eval()
        val_loss = 0.0
        self.metric.reset()
        val_loader_progress = tqdm(
            val_loader,
            desc=f"Validation Epoch {epoch + 1}",
            unit="batch"
        )

        with torch.no_grad():
            for images, targets in val_loader_progress:
                images = [img.to(self.device) for img in images]
                targets = [
                    {k: v.to(self.device) for k, v in t.items()} for t in targets
                ]

                # Compute validation loss
                loss_dict = self.model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                val_loss += losses.item()

                # Compute mAP
                outputs = self.model(images)
                preds = [
                    dict(
                        boxes=output["boxes"].detach().cpu(),
                        scores=output["scores"].detach().cpu(),
                        labels=output["labels"].detach().cpu()
                    )
                    for output in outputs
                ]
                gt_targets = [
                    dict(
                        boxes=t["boxes"].detach().cpu(),
                        labels=t["labels"].detach().cpu()
                    )
                    for t in targets
                ]
                self.metric.update(preds, gt_targets)

                val_loader_progress.set_postfix(loss=f"{losses.item():.4f}")

        avg_val_loss = val_loss / len(val_loader)
        mAP_result = self.metric.compute()
        mAP_score = mAP_result["map"].item()
        return avg_val_loss, mAP_score

    def train(self, train_loader, val_loader):
        """Run the full training loop."""
        print("Learning has kicked off!")
        start_time = datetime.now()
        print(f"Started at: {start_time.strftime('%H:%M:%S')}")

        training_losses = []
        validation_losses = []
        mAP_values = []

        torch.cuda.empty_cache()

        for epoch in range(self.num_epochs):
            print(f"\nEpoch {epoch + 1}/{self.num_epochs} is underway...")
            torch.cuda.empty_cache()

            # Train
            train_loss = self.train_one_epoch(train_loader, epoch)
            training_losses.append(train_loss)

            # Validate
            val_loss, mAP_score = self.validate_one_epoch(val_loader, epoch)
            validation_losses.append(val_loss)
            mAP_values.append(mAP_score)

            # Update learning rate
            self.lr_scheduler.step(val_loss)

            # Log results
            epoch_time = datetime.now()
            print(f"Epoch {epoch + 1}: mAP = {mAP_score:.4f}")
            print(f"Epoch {epoch + 1} ended at: {epoch_time.strftime('%H:%M:%S')}")

        print("Training’s all done!")
        return training_losses, validation_losses, mAP_values

In [30]:
import torch

torch.cuda.empty_cache()
class_names = test_dataset.classes

def test_model_adaboost(models, test_loader, device, class_names, model_weights, visualize=False):
    # Move models to device and set to eval mode
    for model in models:
        model.to(device)
        model.eval()

    # Initialize metrics and accumulators
    metric = MeanAveragePrecision()
    total_test_loss = 0.0

    with torch.no_grad():
        for idxs, images, targets in test_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Collect predictions from all models
            all_predictions = []
            for model in models:
                pred = model(images)
                all_predictions.append(pred)

            # Apply Weighted NMS using AdaBoost weights
            final_preds = weighted_nms(all_predictions, model_weights)

            # Compute loss (optional, only for evaluation)
            total_loss = 0
            for model in models:
                loss_dict = model(images, targets)
                if isinstance(loss_dict, list):
                    losses = sum(sum(loss_value.sum() for loss_value in loss.values()) for loss in loss_dict)
                elif isinstance(loss_dict, dict):
                    losses = sum(loss_value.sum() for loss_value in loss_dict.values())
                else:
                    raise TypeError(f"Unexpected loss format: {type(loss_dict)}")
                total_loss += losses.item()

            total_test_loss += total_loss / len(models)  # Averaging loss over models

            # Convert ground truth to required format for mAP computation
            gt_targets = [
                dict(
                    boxes=t["boxes"].detach().cpu(),
                    labels=t["labels"].detach().cpu()
                ) for t in targets
            ]

            # Update metric
            metric.update(final_preds, gt_targets)

            # Optional: Visualize some predictions
            if visualize:
                visualize_predictions(images, final_preds, class_names)

    # Compute Mean Average Precision (mAP)
    mAP_result = metric.compute()
    mAP_score = mAP_result["map"].item()
    print(f"\nMean Average Precision (mAP): {mAP_score:.4f}")

    # Compute average test loss
    avg_test_loss = total_test_loss / len(test_loader)
    print(f"\nAverage Test Loss: {avg_test_loss:.4f}")

    return mAP_score, avg_test_loss

def visualize_predictions(images, predictions, class_names):

    for i, img in enumerate(images):
        # Denormalize image
        img = denormalize(img.cpu(), mean, std)
        img = F_transform.to_pil_image(img)  # Convert to PIL image

        # Create figure
        fig, ax = plt.subplots(1, figsize=(8, 8))
        ax.imshow(img)

        # Get predicted boxes, labels, and scores
        pred_boxes = predictions[i]['boxes'].cpu().numpy()
        pred_labels = predictions[i]['labels'].cpu().numpy()
        pred_scores = predictions[i]['scores'].cpu().numpy()

        if len(pred_scores) == 0:
            print("No predictions found for this image.")
            continue

        # Find the highest confidence score
        pred_scores_tensor = torch.tensor(pred_scores) 
        max_score_idx = pred_scores_tensor.argmax()  # Get index of highest score

        # Get highest confidence box, label, and score
        best_box = pred_boxes[max_score_idx]
        best_label = pred_labels[max_score_idx]
        best_score = pred_scores[max_score_idx]
        ixd_best_label = pred_labels[max_score_idx].item()
        predicted_label_name = class_names[ixd_best_label - 1]

        # Draw Highest Confidence Bounding Box (Red)
        xmin, ymin, xmax, ymax = best_box
        rect = plt.Rectangle(
            (xmin, ymin), xmax - xmin, ymax - ymin,
            fill=False, color="red", linewidth=2
        )
        ax.add_patch(rect)

        # Display label and score
        ax.text(
            #xmin, ymin - 5, f"{predicted_label_name}: {best_score:.2f}",
            xmin, ymin - 5, f"{predicted_label_name}",
            color="red", fontsize=12, bbox=dict(facecolor='white', alpha=0.7)
        )

        plt.axis("off")
        plt.title("Highest Confidence Prediction")
        plt.show()

In [ ]:
# Run the model on the test dataset
test_model(model, val_loader, device, val_dataset.classes, visualize=True)